In [265]:
import pandas as pd
data = pd.read_csv('PartD_Prescriber_PUF_NPI_17.txt', sep='\t', header=0)
data.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,beneficiary_male_count,beneficiary_race_white_count,beneficiary_race_black_count,beneficiary_race_asian_pi_count,beneficiary_race_hispanic_count,beneficiary_race_nat_ind_count,beneficiary_race_other_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score
0,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,...,92.0,220.0,14.0,0.0,0.0,0.0,0.0,143.0,91.0,2.1685
1,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,...,92.0,195.0,58.0,NaN,NaN,0.0,NaN,143.0,133.0,1.8029
2,1003000167,ESCOBAR,JULIO,E,DDS,M,I,5 PINE CONE RD,NaN,DAYTON,...,17.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,1.0598
3,1003000175,REYES-VASQUEZ,BELINDA,NaN,D.D.S.,F,I,322 N AZUSA AVE STE 202,NaN,LA PUENTE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1003000282,BLAKEMORE,ROSIE,K,FNP,F,I,TENNESSEE PRISON FOR WOMEN,3881 STEWARTS LANE,NASHVILLE,...,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,4.5148


In [266]:
 data["bene_count"].mean()

158.3494585173676

In [267]:
data[data["bene_count"]>=10]["bene_count"].mean()

158.3494585173676

In [268]:
from statistics import median
median(data['total_day_supply']/data['total_claim_count'])

29.7125748502994

In [269]:
data_specialty = data.groupby('specialty_description')['brand_claim_count'].count()
#data_specialty = data_specialty[data_specialty.count()>=1000]
#data_specialty['npi']
#for i in range(len(data_specialty)):
#    if data_specialty[i]<1000:
#        data_specialty.drop(data_specialty.index[i])
#data_specialty
data_specialty[data_specialty>=1000]

specialty_description
Allergy/ Immunology                                                3457
Anesthesiology                                                     3913
Cardiology                                                        16140
Certified Clinical Nurse Specialist                                1785
Clinical Cardiac Electrophysiology                                 1854
Critical Care (Intensivists)                                       1071
Dentist                                                           85976
Dermatology                                                       10574
Diagnostic Radiology                                               1916
Emergency Medicine                                                22349
Endocrinology                                                      5532
Family Practice                                                   76356
Gastroenterology                                                   8665
General Practice                          

In [270]:
#data_specialty.keys()
from statistics import stdev
data_specialty = data.groupby('specialty_description').agg({'brand_claim_count':"count"})
data_frac = data_specialty[data_specialty['brand_claim_count']>=1000]
stdev(data_frac['brand_claim_count']/data_frac['brand_claim_count'].sum())

0.032354117227372595

In [271]:
data_state = data.groupby('nppes_provider_state').agg({'opioid_claim_count':"count",'antibiotic_claim_count':"count"})
data_ratio = data_state['opioid_claim_count']/data_state['antibiotic_claim_count']
max(data_ratio) - min(data_ratio)

0.6785714285714286

In [272]:
from scipy.stats import pearsonr

data_ge65 = data['bene_count_ge65']/data['bene_count']
data_lis = data['lis_claim_count']/data['bene_count']
df = pd.DataFrame(columns = ['data_ge65','data_lis'])
df['data_ge65'] = data_ge65
df['data_lis'] = data_lis
df = df.dropna()
pearsonr(df['data_ge65'],df['data_lis'])

(-0.26764822825768764, 0.0)

In [388]:
data['data_avg_len_opiod'] = data['opioid_day_supply']/data['opioid_claim_count']
#data_alo_ge = data.groupby(['specialty_description','nppes_provider_state']).agg({'npi':'count','data_avg_len_opiod':'mean'})    
#data_alo_ge100 = data_alo_ge[data_alo_ge['npi']>=100]  
#data_alo_ge100
#data_d=data_alo_ge100.to_dict()['data_avg_len_opiod']
#list_key=list(data_d.keys())
#haha=pd.DataFrame(data_d,index=[0])
#haha

#data_d
#data_alo_ge100
data_subset = data.loc[:,['specialty_description','nppes_provider_state','npi','data_avg_len_opiod']]
data_subset['npi_count'] = data_subset.groupby(['specialty_description','nppes_provider_state'])['npi'].transform('count')
data_subset_npi = data_subset[data_subset.loc[:,'npi_count']>=100]

data_subset_npi['len_mean'] = data_subset_npi.groupby(['specialty_description','nppes_provider_state'])['data_avg_len_opiod'].transform('mean')

data_subset_npi['len_mean_spec'] = data_subset_npi.groupby(['specialty_description','nppes_provider_state'])['len_mean'].transform('sum')

#data_subset_npi['data_avg_len_opiod_mean']=data_subset_npi.groupby(['specialty_description'])['data_avg_len_opiod'].transform('mean')
#data_subset_npi['ratio'] = data_subset_npi['data_avg_len_opiod'] / data_subset_npi['data_avg_len_opiod_mean']
#max(data_subset_npi['ratio'])
data_subset_npi['ratio']=data_subset_npi['len_mean'] /data_subset_npi['len_mean_spec'] 
max(data_subset_npi['ratio'])
#data_subset_npi

C:\Users\wuxx1\Softwares\Anacoda\Anaconda3\Python3.7\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\wuxx1\Softwares\Anacoda\Anaconda3\Python3.7\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\wuxx1\Softwares\Anacoda\Anaconda3\Python3.7\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

0.010000000000000021

In [380]:
#table = pd.pivot_table(data, values='data_avg_len_opiod', index=['specialty_description'],
#                   columns=['nppes_provider_state'], aggfunc=np.sum)
#table
#table_new = table.div(table.sum(axis=1),axis=0)
data_alo_ge = data.groupby(['specialty_description','nppes_provider_state']).agg({'npi':'count','data_avg_len_opiod':'mean'})    
data_alo_ge100 = data_alo_ge[data_alo_ge['npi']>=100]  
data_alo_ge100

npi  data_avg_len_opiod
specialty_description nppes_provider_state                         
Allergy/ Immunology   CA                    471           25.636406
                      FL                    225           24.300248
                      IL                    191           18.165693
                      MA                    130           23.956916
                      MI                    144           24.042325
                      NJ                    151           30.717071
                      NY                    365           28.071462
                      OH                    131           16.363384
                      PA                    176           23.450374
                      TX                    309           24.962580
                      VA                    117           10.897727
Anesthesiology        AL                    125           26.228190
                      AZ                    155           25.738227
                      CA                    797           25.728315
                      FL                    493           26.791536
                      GA                    201           27.966697
                      IL                    405           26.710829
                      IN                    159           25.586862
                      KY                    113           27.266909
                      LA                    114           25.509213
                      MA                    259           25.025414
                      MD                    122           25.797571
                      MI                    227           28.294027
                      MO                    183           23.988780
                      NC                    189           28.695493
                      NJ                    243           25.510371
                      NY                    521           26.643820
                      OH                    256           26.774290
                      PA                    325           27.071075
                      SC                    133           26.940023
...                                         ...                 ...
Urology               KY                    146            5.931716
                      LA                    193            6.310056
                      MA                    273            4.137178
                      MD                    231            6.061666
                      MI                    353            5.999182
                      MN                    192            3.796412
                      MO                    206            5.843843
                      NC                    346            5.485811
                      NJ                    352            4.964093
                      NY                    840            5.801978
                      OH                    405            4.945292
                      OK                    120            6.045373
                      OR                    136            4.788823
                      PA                    511            5.313375
                      SC                    163            5.793329
                      TN                    238            4.861296
                      TX                    695            6.085780
                      VA                    250            4.924027
                      WA                    209            4.950667
                      WI                    197            4.568873
Vascular Surgery      CA                    313            6.660592
                      FL                    197            7.121534
                      GA                    117            7.299613
                      IL                    104            8.118733
                      MI                    123            8.177363
                      NJ                    122            6.133482
                      NY        

In [276]:
data16 = pd.read_csv('PartD_Prescriber_PUF_NPI_16.txt', sep='\t', header=0)
data16.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,beneficiary_male_count,beneficiary_race_white_count,beneficiary_race_black_count,beneficiary_race_asian_pi_count,beneficiary_race_hispanic_count,beneficiary_race_nat_ind_count,beneficiary_race_other_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score
0,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,...,80.0,153.0,NaN,0.0,NaN,0.0,NaN,102.0,68.0,2.1950
1,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,...,88.0,159.0,54.0,NaN,NaN,0.0,NaN,108.0,117.0,1.8032
2,1003000167,ESCOBAR,JULIO,E,DDS,M,I,5 PINE CONE RD,NaN,DAYTON,...,12.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,1.0949
3,1003000282,BLAKEMORE,ROSIE,K,FNP,F,I,TENNESSEE PRISON FOR WOMEN,3881 STEWARTS LANE,NASHVILLE,...,11.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,1.7014
4,1003000407,GIRARDI,DAVID,J,D.O.,M,I,100 HOSPITAL RD,NaN,BROOKVILLE,...,64.0,NaN,NaN,0.0,0.0,0.0,NaN,129.0,61.0,1.9105


In [285]:
data16['cost16'] = data16['total_drug_cost']/data16['total_day_supply']
data16_sub = data16.loc[:,['npi','cost16']]
data['cost17'] = data['total_drug_cost']/data['total_day_supply']
data_sub = data.loc[:,['npi','cost17']]

In [293]:
df = data16_sub.set_index('npi').join(data_sub.set_index('npi'))
(df['cost17']/df['cost16']).mean()

1.2274706790253758

In [376]:
data16_subset = data16.loc[:,['specialty_description','npi']]
data16_subset.rename(columns={'specialty_description':'specialty_description16'},inplace = True)
data16_subset['npi_count16']=data16_subset.groupby('specialty_description16')['npi'].transform('count')
data16_subset=data16_subset[data16_subset['npi_count16']>=1000]
data17_subset = data.loc[:,['specialty_description','npi']]
data17_subset.rename(columns={"specialty_description":"specialty_description17" },inplace = True)
#print(data17_subset)
data17_subset['npi_count17']=data17_subset.groupby('specialty_description17')['npi'].transform('count')
data17_subset=data17_subset[data17_subset['npi_count17']>=1000]
df3 = data16_subset.set_index('npi').join(data17_subset.set_index('npi'))



138.58658204895738

In [378]:
max((df3['npi_count16']-df3['npi_count17'])/df3['npi_count16'])

0.9919203448120504